In [ ]:
import numpy as np
import pandas as pd
import warnings
import joblib
import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import optuna

RANDOM_SEED = 0

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)
np.random.seed(RANDOM_SEED)

# =============================================================================
# CONFIGURATION
# =============================================================================
print("="*80)
print("SVR HYPERPARAMETER OPTIMIZATION")
print("="*80)

# =============================================================================
# LOAD PREPROCESSED DATA
# =============================================================================
print("\n" + "="*80)
print("LOADING PREPROCESSED DATA")
print("="*80)

X_train = joblib.load('../../data/preprocessed/X_train.pkl')
y_train = joblib.load('../../data/preprocessed/y_train.pkl')
X_test = joblib.load('../../data/preprocessed/X_test.pkl')
metadata = joblib.load('../../data/preprocessed/metadata.pkl')

print(f"✓ Train shape: {X_train.shape}")
print(f"✓ Target shape: {y_train.shape}")
print(f"✓ Test shape: {X_test.shape}")

# =============================================================================
# OPTIMIZATION SETTINGS
# =============================================================================
FAST_MODE = False  # Set to False for more thorough search
N_TRIALS = 15 if FAST_MODE else 100
N_FOLDS = 5
N_JOBS_OPTUNA = 2  # SVR doesn't parallelize well, use fewer jobs
SAMPLE_SIZE = 20000  # Subsample for speed (SVR is O(n²) to O(n³))

print(f"\n{'='*60}")
print(f"OPTIMIZATION SETTINGS")
print(f"{'='*60}")
print(f"Mode: {'FAST' if FAST_MODE else 'THOROUGH'}")
print(f"Trials: {N_TRIALS}")
print(f"Folds: {N_FOLDS}")
print(f"Parallel jobs: {N_JOBS_OPTUNA}")
print(f"Sample size per fold: {SAMPLE_SIZE}")
print(f"⚠️  Note: Using subsampling due to SVR's computational complexity")

# =============================================================================
# OBJECTIVE FUNCTION
# =============================================================================
def objective_svr(trial, X, y):
    """Objective function for SVR optimization"""
    
    # Hyperparameters to optimize
    params = {
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'linear', 'poly']),
        'C': trial.suggest_float('C', 0.1, 100, log=True),
        'epsilon': trial.suggest_float('epsilon', 0.01, 1.0, log=True),
    }
    
    # Kernel-specific parameters
    if params['kernel'] == 'rbf':
        params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])
    elif params['kernel'] == 'poly':
        params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])
        params['degree'] = trial.suggest_int('degree', 2, 5)
        params['coef0'] = trial.suggest_float('coef0', 0.0, 10.0)

    scaler = StandardScaler()
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)
    scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        
        # Handle both pandas Series and numpy arrays
        if isinstance(y, pd.Series):
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        else:
            y_tr, y_val = y[train_idx], y[val_idx]

        # Scale the data (CRITICAL for SVR)
        X_tr_scaled = scaler.fit_transform(X_tr)
        X_val_scaled = scaler.transform(X_val)

        # Subsample for speed (SVR is O(n²) to O(n³))
        if len(X_tr) > SAMPLE_SIZE:
            sample_idx = np.random.choice(len(X_tr), SAMPLE_SIZE, replace=False)
            X_tr_scaled = X_tr_scaled[sample_idx]
            y_tr = y_tr.iloc[sample_idx] if isinstance(y_tr, pd.Series) else y_tr[sample_idx]

        # Train SVR
        model = SVR(**params)
        
        try:
            model.fit(X_tr_scaled, y_tr)
            preds = np.clip(model.predict(X_val_scaled), 0, 100)
            rmse = np.sqrt(mean_squared_error(y_val, preds))
            scores.append(rmse)
        except Exception as e:
            # If model fails, return high penalty
            return 1e6

    return np.mean(scores)

# =============================================================================
# RUN OPTIMIZATION
# =============================================================================
print("\n" + "="*80)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*80)

start_time = time.time()

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
)

study.optimize(
    lambda trial: objective_svr(trial, X_train, y_train),
    n_trials=N_TRIALS,
    show_progress_bar=True,
    n_jobs=N_JOBS_OPTUNA
)

optimization_time = time.time() - start_time

# =============================================================================
# RESULTS
# =============================================================================
print("\n" + "="*80)
print("OPTIMIZATION RESULTS")
print("="*80)
print(f"Best RMSE: {study.best_value:.6f}")
print(f"Optimization time: {optimization_time:.1f}s")
print(f"\nBest parameters:")
for param, value in sorted(study.best_params.items()):
    print(f"  {param:20s}: {value}")

# =============================================================================
# SAVE RESULTS
# =============================================================================
# Save best parameters
best_params = study.best_params.copy()

joblib.dump(best_params, 'svr_params.pkl')
print("\n✓ Parameters saved to: svr_params.pkl")

# Save optimization history
history = pd.DataFrame({
    'trial': [t.number for t in study.trials],
    'value': [t.value for t in study.trials],
    'params': [str(t.params) for t in study.trials]
})
history.to_csv('svr_history.csv', index=False)
print("✓ History saved to: svr_history.csv")

# Save study object
joblib.dump(study, 'svr_study.pkl')
print("✓ Study saved to: svr_study.pkl")

# Save summary
summary = {
    'model': 'SVR',
    'best_rmse': study.best_value,
    'n_trials': N_TRIALS,
    'n_folds': N_FOLDS,
    'optimization_time': optimization_time,
    'sample_size': SAMPLE_SIZE,
    'best_params': best_params
}
joblib.dump(summary, 'svr_summary.pkl')
print("✓ Summary saved to: svr_summary.pkl")

# =============================================================================
# TRAIN FINAL MODEL ON FULL TRAINING DATA
# =============================================================================
print("\n" + "="*80)
print("TRAINING FINAL MODEL ON FULL DATASET")
print("="*80)

# Scale the entire training set
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Optionally subsample for final training (if dataset is very large)
use_full_data = len(X_train) <= 50000
if use_full_data:
    print(f"Training on full dataset ({len(X_train)} samples)...")
    X_final = X_train_scaled
    y_final = y_train
else:
    final_sample_size = 50000
    print(f"⚠️  Dataset too large for SVR, subsampling to {final_sample_size} samples...")
    sample_idx = np.random.choice(len(X_train), final_sample_size, replace=False)
    X_final = X_train_scaled[sample_idx]
    y_final = y_train.iloc[sample_idx] if isinstance(y_train, pd.Series) else y_train[sample_idx]

final_model = SVR(**best_params)
print("Training final model...")
final_model.fit(X_final, y_final)

# Save final model and scaler
joblib.dump(final_model, 'svr_final_model.pkl')
joblib.dump(scaler, 'svr_scaler.pkl')
print("\n✓ Final model saved to: svr_final_model.pkl")
print("✓ Scaler saved to: svr_scaler.pkl")

# =============================================================================
# GENERATE PREDICTIONS AND SUBMISSION FILE
# =============================================================================
print("\n" + "="*80)
print("GENERATING PREDICTIONS")
print("="*80)

# Scale test data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Make predictions on test set
test_predictions = final_model.predict(X_test_scaled)

# Clip predictions to valid range [0, 100]
test_predictions = np.clip(test_predictions, 0, 100)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': range(len(X_train), len(test_predictions) + len(X_train)),
    'test_score': test_predictions
})

# Save submission file
submission.to_csv('submission.csv', index=False)
print("✓ Submission file saved to: submission.csv")

print(f"\nSubmission statistics:")
print(f"  Min prediction: {test_predictions.min():.2f}")
print(f"  Max prediction: {test_predictions.max():.2f}")
print(f"  Mean prediction: {test_predictions.mean():.2f}")
print(f"  Median prediction: {np.median(test_predictions):.2f}")
print(f"  Std prediction: {test_predictions.std():.2f}")
print(f"  Shape: {submission.shape}")

# =============================================================================
# MODEL STATISTICS
# =============================================================================
print("\n" + "="*80)
print("MODEL STATISTICS")
print("="*80)
print(f"Kernel: {best_params['kernel']}")
print(f"C (regularization): {best_params['C']:.4f}")
print(f"Epsilon (tube width): {best_params['epsilon']:.4f}")
if 'gamma' in best_params:
    print(f"Gamma: {best_params['gamma']}")
if 'degree' in best_params:
    print(f"Polynomial degree: {best_params['degree']}")
print(f"Number of support vectors: {len(final_model.support_vectors_)}")
print(f"Support vector ratio: {len(final_model.support_vectors_) / len(X_final) * 100:.2f}%")

# =============================================================================
# VISUALIZATIONS
# =============================================================================
print("\n" + "="*80)
print("GENERATING VISUALIZATIONS")
print("="*80)

import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Optimization history
ax1 = axes[0, 0]
history = pd.read_csv('svr_history.csv')
ax1.plot(history['trial'], history['value'], alpha=0.6, marker='o', markersize=4)
ax1.plot(history['trial'], history['value'].cummin(), 
         linewidth=2.5, label='Best score', color='red')
ax1.set_xlabel('Trial', fontsize=11, fontweight='bold')
ax1.set_ylabel('RMSE', fontsize=11, fontweight='bold')
ax1.set_title('Optimization Progress', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Kernel comparison (if multiple kernels were tried)
ax2 = axes[0, 1]
kernel_counts = {}
kernel_scores = {}
for t in study.trials:
    if 'kernel' in t.params:
        kernel = t.params['kernel']
        kernel_counts[kernel] = kernel_counts.get(kernel, 0) + 1
        if kernel not in kernel_scores:
            kernel_scores[kernel] = []
        kernel_scores[kernel].append(t.value)

if kernel_scores:
    kernels = list(kernel_scores.keys())
    avg_scores = [np.mean(kernel_scores[k]) for k in kernels]
    
    bars = ax2.bar(kernels, avg_scores, color=['skyblue', 'lightcoral', 'lightgreen'][:len(kernels)],
                   edgecolor='black', linewidth=1.5)
    ax2.set_ylabel('Average RMSE', fontsize=11, fontweight='bold')
    ax2.set_title('Kernel Performance Comparison', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    for bar, val in zip(bars, avg_scores):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.4f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
else:
    ax2.text(0.5, 0.5, 'Single kernel used', ha='center', va='center',
            transform=ax2.transAxes, fontsize=12)

# 3. Support vectors visualization (sample)
ax3 = axes[1, 0]
sv_ratio = len(final_model.support_vectors_) / len(X_final) * 100
labels = ['Support Vectors', 'Regular Points']
sizes = [sv_ratio, 100 - sv_ratio]
colors = ['#ff9999', '#66b3ff']
explode = (0.1, 0)

ax3.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=90)
ax3.set_title(f'Support Vector Distribution\n({len(final_model.support_vectors_)} SVs)', 
              fontsize=12, fontweight='bold')

# 4. Prediction distribution
ax4 = axes[1, 1]
ax4.hist(test_predictions, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax4.axvline(test_predictions.mean(), color='red', linestyle='--', 
           linewidth=2, label=f'Mean: {test_predictions.mean():.2f}')
ax4.axvline(np.median(test_predictions), color='green', linestyle='--',
           linewidth=2, label=f'Median: {np.median(test_predictions):.2f}')
ax4.set_xlabel('Predicted Score', fontsize=11, fontweight='bold')
ax4.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax4.set_title('Test Predictions Distribution', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('svr_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Analysis plot saved to: svr_analysis.png")
plt.show()

# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("\n" + "="*80)
print("SVR OPTIMIZATION COMPLETE!")
print("="*80)
print(f"✓ Best CV RMSE: {study.best_value:.6f}")
print(f"✓ Final model trained on {len(X_final)} samples")
print(f"✓ Number of support vectors: {len(final_model.support_vectors_)}")
print(f"✓ Predictions generated for {len(X_test)} test samples")
print(f"✓ All results saved")
print("\nFiles created:")
print("  • svr_params.pkl")
print("  • svr_history.csv")
print("  • svr_study.pkl")
print("  • svr_summary.pkl")
print("  • svr_final_model.pkl")
print("  • svr_scaler.pkl")
print("  • svr_analysis.png")
print("  • submission.csv")
print("\n⚠️  Important Notes:")
print("  - SVR requires scaled data (StandardScaler used)")
print("  - Subsampling was used due to computational complexity")
print("  - For large datasets, consider using LinearSVR or other models")
print("="*80)

SVR HYPERPARAMETER OPTIMIZATION

LOADING PREPROCESSED DATA
✓ Train shape: (630000, 9)
✓ Target shape: (630000,)

OPTIMIZATION SETTINGS
Mode: FAST
Trials: 5
Folds: 5

STARTING BAYESIAN OPTIMIZATION
Note: Using data sampling for speed (SVR is computationally intensive)


  0%|          | 0/5 [00:00<?, ?it/s]